症状推荐症状模型开发

In [1]:
import sys
sys.path.append('/home/ian/code/github')

### 查看现在疾病库中典型症状有多少个

In [2]:
from utils.pymongo_wrapper import pymongo_wrapper as pmw

from utils.pickle_wrapper import pickle_wrapper as pkw

pmw = pmw()

distable = pmw.get_collection('jiankang39','diseases')

zzl = list(pmw.find_all(distable,fieldlist=['_id','典型症状']))

zzset = set()
for z in zzl:
    if '典型症状' in z:
        zzset = zzset.union(z['典型症状'])
len(zzset)#4138

4138

### 症状库中有多少症状

In [4]:
zztable = pmw.get_collection('jiankang39','zznew')

zz2 = list(pmw.find_all(zztable,fieldlist=['原名称']))

zzlist2 = [i['原名称'] for i in zz2]
len(zzlist2)#5898

3488

### 取并集

In [6]:
#共有症状
zzshared = list(zzset.intersection(set(zzlist2))) 
len(zzshared)#3633

2304

## 构建共现矩阵

In [7]:
# import numpy as np

# tongxianmatr = np.zeros((len(zzshared),len(zzshared)))

# import pandas as pd

# zzshared_series = pd.Series(zzshared)

# zzshared_series[:5]

# zzshared_series_inv = pd.Series(zzshared_series.index,index=zzshared_series.values)
# zzshared_series_inv[:5]

# pkw.dump2file(zzshared_series,'共有症状Series.pkl')
# pkw.dump2file(zzshared_series_inv,'共有症状逆序Series.pkl')
zzshared_series = pkw.loadfromfile('共有症状Series.pkl')
zzshared_series_inv = pkw.loadfromfile('共有症状逆序Series.pkl')

In [8]:
l1 = list()
for z in zzl:
    if '典型症状' in z:
        t = [zzshared_series_inv[i] for i in z['典型症状'] if i in zzshared_series_inv]
        if len(t)>1:
            l1.append(t)
l1[:2]#[[3479, 1494, 1120], [3629, 578, 2054]]

[[922, 1443, 1307], [2014, 82]]

In [9]:
# import itertools

# l2 = list()
# for i in l1:
#     for ii in i:
#         l2.append((ii,ii))
#     l2.extend(list(itertools.permutations(i,2)))

# l2[:5]#[(3479, 3479), (1494, 1494), (1120, 1120), (3479, 1494), (3479, 1120)]

# for i in l2:
#     tongxianmatr[i] = tongxianmatr[i]+1

# pkw.dump2file(tongxianmatr,'/home/ian/code/data/daozhen/同现矩阵ndarray.pkl')
tongxianmatr = pkw.loadfromfile('/home/ian/code/data/daozhen/同现矩阵ndarray.pkl')

In [10]:
tongxianmatr.shape

(2304, 2304)

In [11]:
#为了避免推荐出不可推荐的症状，这里也需要传入专门的zzlist
#但是这样做可能会是计算变慢，最好提前根据分类计算好对应的zzlist
def zz2zz(zzlist1):
    t = [zzshared_series_inv[i] for i in zzlist1 if i in zzshared_series_inv]
    if t:
        r = None
        for i in t:
            if not r:
                r = tongxianmatr[i]
            else:
                r = r + tongxianmatr[i]
            s = pd.Series(list(r)).sort_values(ascending=False)
            l = list(s[s>0].index)
            l1 = [i for i in l if i not in t]
            if l1:
                z = [zzshared_series[i] for i in l1]
                return z,l1,[s[i] for i in l1]

In [30]:
zzshared_series[2659],zzshared_series[2648],zzshared_series[2304],zzshared_series[3386],zzshared_series[2281],zzshared_series[2234]

('耳垂长痘', '大便黑色', '逼尿肌无反射', '小腿瘙痒干燥', '尿道的条索状肿块', '小脑性共济失调')

In [12]:
zz2zz(['单侧头痛'])

(['两眼下视不能', '头昏', '头胀', '眼肌麻痹', '电击样痛或刺痛的头痛', '头痛为搏动性'],
 [1602, 746, 57, 2037, 1443, 1307],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [13]:
zz2zz(['头胀'])

(['头痛',
  '血压高',
  '头晕',
  '头昏',
  '中度或轻度头痛',
  '乏力',
  '听力减退',
  '发作性眩晕',
  '单侧头痛',
  '易激动',
  '健忘'],
 [1196, 937, 306, 746, 339, 1811, 2200, 374, 922, 542, 1917],
 [4.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [14]:
zz2zz(['头胀','单侧头痛'])

(['头痛', '血压高', '头晕', '头昏', '中度或轻度头痛', '乏力', '听力减退', '发作性眩晕', '易激动', '健忘'],
 [1196, 937, 306, 746, 339, 1811, 2200, 374, 542, 1917],
 [4.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [81]:
zz2zz(['头痛','头胀','单侧头痛', '乏力', '血压高', '头昏'])

(['发烧',
  '头晕',
  '恶心与呕吐',
  '恶心',
  '视力障碍',
  '咽痛',
  '肚子疼',
  '鼻塞',
  '眼痛',
  '肌痛',
  '怕冷',
  '水肿',
  '颅内压增高',
  '关节痛',
  '咳嗽',
  '食欲减退',
  '烦躁不安',
  '嗜睡',
  '脓性分泌物',
  '无力',
  '抽搐',
  '失眠',
  '胸痛',
  '共济失调',
  '癫痫和癫痫样发作',
  '疼痛',
  '呼吸困难',
  '疲乏',
  '肌肉酸痛',
  '感染性发热',
  '溢乳',
  '昏迷',
  '恶寒',
  '低烧',
  '食欲不振',
  '胸闷',
  '气短',
  '消瘦',
  '耳痛',
  '疲劳',
  '神经痛',
  '高热寒战',
  '糖尿',
  '斑丘疹',
  '视网膜脱离',
  '耳流脓',
  '偏瘫',
  '下肢神经性疼痛',
  '视力视野改变',
  '全身感染中毒症状',
  '高眼压',
  '高热',
  '目赤',
  '流清涕',
  '周身不适',
  '全头痛',
  '带血粘脓鼻涕',
  '复视',
  '心悸',
  '胁肋灼痛',
  '步态异常',
  '眼球不能随意动',
  '关节肿痛',
  '鼻甲肿胀',
  '打呼噜',
  '尿磷',
  '流鼻血',
  '肝脾肿大',
  '两眼干涩',
  '视野缺损',
  '纳差',
  '垂体功能减退',
  '面红升火',
  '深部性头痛',
  '颈根部斜方肌及风池穴处有压痛点',
  '腹股沟淋巴结肿大',
  '流黄鼻涕',
  '虹视',
  '闭经-乳溢-不育三联征',
  '意识模糊',
  '全身各脏器血流缓慢',
  '浑身酸痛',
  '谵妄',
  '息肉',
  '发绀',
  '皮肤黑斑',
  '组织液渗出',
  '意识朦胧状态',
  '导水管周围受损',
  '足趾发凉怕冷，呈苍白色或紫红色',
  '视物变形',
  '神经衰弱综合征',
  '剧痛',
  '散脉',
  '鼻衄',
  '眼胀',
  '脉浮紧',
  '关节疼痛',
  '男子性功能障碍',
  '发作性咳嗽',
  '消化不良',
  '气急',
